In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 18 17:06:31 2019

@author: mor
"""



In [ ]:


from keras.layers import Input, Dense, Embedding, Flatten, Concatenate
import pandas as pd
from keras.utils import plot_model
import matplotlib.pyplot as plt
from numpy import unique
from keras.models import Model



In [ ]:

games_season=pd.read_csv('data/games_season.csv')
games_tourney = pd.read_csv('data/games_tourney.csv')


In [ ]:


# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')



In [ ]:


# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)



In [ ]:


# Import the model class
from keras.models import Model

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')



In [ ]:


# Fit the model to the games_season dataset
model.fit([games_season['team_1'], games_season['team_2'], games_season['home']],
          games_season['score_diff'],
          epochs=1,
          verbose=True,
          validation_split=0.1,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
model.evaluate([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']],
          games_tourney['score_diff'])




In [ ]:

# Imports
import matplotlib.pyplot as plt
from keras.utils import plot_model

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()



In [ ]:

# =============================================================================
#*** stacking ***
# using train model prediction and add the value to games_tourney data = pred
# afterward using the new and old data of games_tourney to build new model
# =============================================================================

# Predict
games_tourney['pred'] = model.predict([games_tourney['team_1'],games_tourney['team_2'],games_tourney['home']])



In [ ]:


# Create an input layer with 3 columns
input_tensor = Input((3,))

# Pass it to a Dense layer with 1 unit
output_tensor = Dense(1)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')



In [ ]:

games_tourney_train=games_tourney.iloc[:3618]
games_tourney_test=games_tourney.iloc[3618:]




In [ ]:

# Fit the model
model.fit(games_tourney_train[['home', 'seed_diff', 'pred']],
          games_tourney_train['score_diff'],
          epochs=1,
          verbose=True)



In [ ]:


# Evaluate the model on the games_tourney_test dataset
model.evaluate(games_tourney_test[['home','seed_diff','pred']], 
               games_tourney_test['score_diff'])

